# Transfer learning: Resnet-152

## The code that produces the .csv file is at the very bottom.

In [23]:
import torch
import numpy as np

from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.autograd import Variable
import PIL.Image as Image
from torchvision import models
import time
import os

In [24]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

train_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
        ])

valid_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
            normalize,
        ])


train_dataset = ImageFolder(
        root='./bird_dataset/train_images/', 
        transform=train_transform
    )

valid_dataset = ImageFolder(
        root='./bird_dataset/val_images/', 
        transform=valid_transform
    )


In [25]:
batch_size = 16
num_workers = 1  #1 if cuda
pin_memory = True  #true if cuda
shuffle = True

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

In [27]:
cuda_avail = torch.cuda.is_available()

model_ft = models.resnet152(pretrained=True) 
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 20)     
 
if cuda_avail:                                 
    model_ft = model_ft.cuda()       

loss_fn = nn.CrossEntropyLoss()          
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#inspired from
# https://heartbeat.fritz.ai/basics-of-image-classification-with-pytorch-2f8973c51864


def save_models(epoch):
    torch.save(model_ft.state_dict(), "resnetmodel_{}.model".format(epoch))
    print("Checkpoint saved")

def test():
    model_ft.eval()
    test_acc = 0.0
    for data in valid_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model_ft(images)
        _,prediction = torch.max(outputs.data, 1)
        test_acc += (prediction == labels).sum().item()

    test_acc = test_acc / 103

    return test_acc

best_acc_train = 0.0
best_acc_test = 0.0

def train(num_epochs):
    global best_acc_train
    global best_acc_test
    for epoch in range(num_epochs):
        model_ft.train()
        train_acc = 0.0
        train_loss = 0.0
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model_ft(images)
            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.cpu().item() * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += (prediction == labels).sum().item()

        train_acc = train_acc / 1082
        train_loss = train_loss / 1082

        test_acc = test()

        if (test_acc >= best_acc_test) and (train_acc >= best_acc_train):
            save_models(epoch)
            best_acc_test = test_acc
            best_acc_train = train_acc


        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))


In [9]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.8438077634011091 , TrainLoss: 0.5333780475552994 , Test Accuracy: 0.8932038834951457
Epoch 1, Train Accuracy: 0.8678373382624769 , TrainLoss: 0.43252752591854104 , Test Accuracy: 0.883495145631068
Epoch 2, Train Accuracy: 0.8696857670979667 , TrainLoss: 0.44002513711421637 , Test Accuracy: 0.8737864077669902
Epoch 3, Train Accuracy: 0.8622920517560074 , TrainLoss: 0.4640106884917578 , Test Accuracy: 0.9029126213592233
Epoch 4, Train Accuracy: 0.8798521256931608 , TrainLoss: 0.41491544406427255 , Test Accuracy: 0.8932038834951457


In [10]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.8807763401109058 , TrainLoss: 0.39712828211830636 , Test Accuracy: 0.912621359223301
Checkpoint saved
Epoch 1, Train Accuracy: 0.8853974121996303 , TrainLoss: 0.39955970534775925 , Test Accuracy: 0.9320388349514563
Epoch 2, Train Accuracy: 0.8770794824399261 , TrainLoss: 0.4009243851172946 , Test Accuracy: 0.9223300970873787
Epoch 3, Train Accuracy: 0.8743068391866913 , TrainLoss: 0.39618106271775505 , Test Accuracy: 0.9029126213592233
Epoch 4, Train Accuracy: 0.8881700554528651 , TrainLoss: 0.3690962738558898 , Test Accuracy: 0.8640776699029126


In [11]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.899260628465804 , TrainLoss: 0.3601100623827985 , Test Accuracy: 0.8543689320388349
Epoch 1, Train Accuracy: 0.8974121996303143 , TrainLoss: 0.333515292147171 , Test Accuracy: 0.9029126213592233
Epoch 2, Train Accuracy: 0.899260628465804 , TrainLoss: 0.3559979760051875 , Test Accuracy: 0.8932038834951457
Epoch 3, Train Accuracy: 0.9001848428835489 , TrainLoss: 0.355687818939276 , Test Accuracy: 0.8932038834951457
Epoch 4, Train Accuracy: 0.9066543438077634 , TrainLoss: 0.3274233182014247 , Test Accuracy: 0.883495145631068


In [12]:
optimizer = SGD(model_ft.parameters(), lr=0.00001, momentum=0.9)
train(20)

Epoch 0, Train Accuracy: 0.910351201478743 , TrainLoss: 0.29681951545743535 , Test Accuracy: 0.912621359223301
Epoch 1, Train Accuracy: 0.9020332717190388 , TrainLoss: 0.310364294074159 , Test Accuracy: 0.8932038834951457
Epoch 2, Train Accuracy: 0.9094269870609981 , TrainLoss: 0.30887515088216216 , Test Accuracy: 0.8932038834951457
Epoch 3, Train Accuracy: 0.9168207024029574 , TrainLoss: 0.28270128564891883 , Test Accuracy: 0.9029126213592233
Epoch 4, Train Accuracy: 0.9085027726432532 , TrainLoss: 0.31333437137815295 , Test Accuracy: 0.912621359223301
Epoch 5, Train Accuracy: 0.9158964879852126 , TrainLoss: 0.2853762205008438 , Test Accuracy: 0.912621359223301
Epoch 6, Train Accuracy: 0.9158964879852126 , TrainLoss: 0.2994083102220969 , Test Accuracy: 0.912621359223301
Epoch 7, Train Accuracy: 0.9288354898336414 , TrainLoss: 0.24886606953170515 , Test Accuracy: 0.883495145631068
Epoch 8, Train Accuracy: 0.9149722735674677 , TrainLoss: 0.2810341846920867 , Test Accuracy: 0.89320388349

In [14]:
optimizer = SGD(model_ft.parameters(), lr=0.0000001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.9177449168207024 , TrainLoss: 0.28156084284985133 , Test Accuracy: 0.912621359223301
Epoch 1, Train Accuracy: 0.922365988909427 , TrainLoss: 0.2901320512270531 , Test Accuracy: 0.9029126213592233
Epoch 2, Train Accuracy: 0.9168207024029574 , TrainLoss: 0.27392545642672095 , Test Accuracy: 0.9029126213592233
Epoch 3, Train Accuracy: 0.9121996303142329 , TrainLoss: 0.2900943620726053 , Test Accuracy: 0.8932038834951457
Epoch 4, Train Accuracy: 0.9260628465804066 , TrainLoss: 0.25716944146288523 , Test Accuracy: 0.8932038834951457


In [16]:
optimizer = SGD(model_ft.parameters(), lr=0.0000001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.9149722735674677 , TrainLoss: 0.2926082981340656 , Test Accuracy: 0.912621359223301
Epoch 1, Train Accuracy: 0.9121996303142329 , TrainLoss: 0.2852314214829817 , Test Accuracy: 0.912621359223301
Epoch 2, Train Accuracy: 0.9158964879852126 , TrainLoss: 0.2903482534693702 , Test Accuracy: 0.9029126213592233
Epoch 3, Train Accuracy: 0.9325323475046211 , TrainLoss: 0.2577212348222292 , Test Accuracy: 0.912621359223301
Epoch 4, Train Accuracy: 0.9085027726432532 , TrainLoss: 0.30452966904904616 , Test Accuracy: 0.9029126213592233


In [18]:
optimizer = SGD(model_ft.parameters(), lr=0.0000001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.9131238447319778 , TrainLoss: 0.2857175715088624 , Test Accuracy: 0.9029126213592233
Epoch 1, Train Accuracy: 0.9094269870609981 , TrainLoss: 0.30698488132129537 , Test Accuracy: 0.8932038834951457
Epoch 2, Train Accuracy: 0.9020332717190388 , TrainLoss: 0.3146503059513688 , Test Accuracy: 0.8932038834951457
Epoch 3, Train Accuracy: 0.9242144177449169 , TrainLoss: 0.26262853901950356 , Test Accuracy: 0.8932038834951457
Epoch 4, Train Accuracy: 0.9048059149722736 , TrainLoss: 0.3003206422721172 , Test Accuracy: 0.883495145631068


In [19]:
optimizer = SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.9149722735674677 , TrainLoss: 0.27921398905365835 , Test Accuracy: 0.883495145631068
Epoch 1, Train Accuracy: 0.9094269870609981 , TrainLoss: 0.32071738498708896 , Test Accuracy: 0.912621359223301
Checkpoint saved
Epoch 2, Train Accuracy: 0.9205175600739371 , TrainLoss: 0.28127728990673356 , Test Accuracy: 0.9029126213592233
Checkpoint saved
Epoch 3, Train Accuracy: 0.9214417744916821 , TrainLoss: 0.2872292442236943 , Test Accuracy: 0.912621359223301
Epoch 4, Train Accuracy: 0.9131238447319778 , TrainLoss: 0.2882577839433155 , Test Accuracy: 0.9029126213592233


In [21]:
model_ft.load_state_dict(torch.load('./resnetmodel_3.model'))

In [22]:
test_dir = './bird_dataset/test_images/mistery_category'
output_file = open("test2.csv", "w")
output_file.write("Id,Category\n")

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

for f in os.listdir(test_dir):
    if 'jpg' in f:
        data = valid_transform(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if cuda_avail:
            data = data.cuda()
        model_ft.eval()
        output = model_ft(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

In [26]:
torch.save(model_ft.state_dict(), "resnet_20epochs_0.81290.model")

In [34]:
torch.save(model_ft.state_dict(), "resnet_xepochs_0.83225.model")

In [13]:
torch.save(model_ft.state_dict(), "resnet_0.29480_.model")